In [1]:
from pathlib import Path
import numpy as np
import os
import sys
import pickle
import random
import torch
import torch.nn as nn
import torch.nn.functional as F

import imblearn

from numpy import where
from matplotlib import pyplot
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.datasets import make_classification



## Load numpy arrays from the path where process icustays program saved those arrays

In [2]:
path = Path('/finalproject/npoutput').expanduser()

chart_data_np = np.load(path/'chart_all_data_04272109.npy')
demographic_data_np = np.load(path/'demographic_all_data_04272109.npy')
icd9_data_np = np.load(path/'icd9_all_data_04272109.npy')
y_np = np.load(path/'y_all_04272109.npy')
    
#chart_data_np = np.load(path/'chart_sample_data_04252037.npy')
#demographic_data_np = np.load(path/'demographic_sample_data_04252037.npy')
#icd9_data_np = np.load(path/'icd9_sample_data_04252037.npy')
#y_np = np.load(path/'y_04252037.npy')


print(chart_data_np.shape)
print(demographic_data_np.shape)
print(icd9_data_np.shape)
print(y_np.shape)

(48854, 48, 59)
(48854, 14)
(48854, 17)
(48854,)


## Create a new numpy array concatenating chart events, icd9 groups and patient demographics

In [3]:
new_chart_demo_icd9 = np.zeros((chart_data_np.shape[0], chart_data_np.shape[1], chart_data_np.shape[2] + demographic_data_np.shape[1] + icd9_data_np.shape[1]))

print(new_chart_demo_icd9.shape)

#new_chart_demo[:,:,0:58] = chart_data_np

demographic_data_np_new = np.zeros((demographic_data_np.shape[0],48,demographic_data_np.shape[1]))
icd9_new = np.zeros((icd9_data_np.shape[0],48,icd9_data_np.shape[1]))

for i in range(demographic_data_np.shape[0]):
    for j in range(48):
        demographic_data_np_new[i,j,:] = demographic_data_np[i, :]
    

for k in range(icd9_data_np.shape[0]):
    for l in range(48):
        icd9_new[k,l,:] = icd9_data_np[k, :]
        

print(new_chart_demo_icd9.shape)
new_chart_demo_icd9[:,:,0:59] = chart_data_np
new_chart_demo_icd9[:,:,59:73] = demographic_data_np_new
new_chart_demo_icd9[:,:,73:91] = icd9_new


(48854, 48, 90)
(48854, 48, 90)


## Oversample positive icustays as there is data imbalance between positive and negative icustays in the original data

In [4]:
counter = Counter(y_np)
print(counter)

#chart_data_transposed = chart_data_np.transpose(2,0,1).reshape(3,-1)
chart_data_np_reshape = new_chart_demo_icd9.reshape(len(new_chart_demo_icd9),-1)
print(chart_data_np_reshape.shape)

print(y_np.shape)
oversample = SMOTE()
chart_data_np_reshape, y_np1 = oversample.fit_resample(chart_data_np_reshape, y_np)
# summarize the new class distribution
chart_data_np_1 = chart_data_np_reshape.reshape(len(chart_data_np_reshape),48,90)
counter = Counter(y_np1)
print(chart_data_np.shape)
print(y_np1.shape)
print(counter)


Counter({0.0: 39227, 1.0: 9627})
(48854, 4320)
(48854,)
(48854, 48, 59)
(78454,)
Counter({0.0: 39227, 1.0: 39227})


## Convert numpy arrays to pytorch arrays

In [5]:
chart_data = torch.from_numpy(chart_data_np_1).float()
demographic_data = torch.from_numpy(demographic_data_np).float()
icd9_data = torch.from_numpy(icd9_data_np).float()
y = torch.from_numpy(y_np1).float()


## Create CustomDataset class

In [6]:
#Prepare Dataset
from torch.utils.data import Dataset


class CustomDataset(Dataset):
    
    def __init__(self, x, y):
        
        self.x = x
        self.y = y
    
    def __len__(self):
        
        return len(self.x)
    
    def __getitem__(self, index):
        
        return self.x[index], self.y[index]
        

dataset = CustomDataset(chart_data, y)



In [7]:
print(dataset)



## Collate function to collect batch data

In [8]:
def collate_fn(data):
    #print(len(data))
    x, y = zip(*data)
    return torch.stack(x), torch.stack(y).type(torch.LongTensor) 



## Split dataset into 80% training, 10% validation and 10% testing

In [9]:
from torch.utils.data.dataset import random_split

split = int(len(dataset)*0.8)

lengths = [split, len(dataset) - split]
train_dataset, rem_dataset = random_split(dataset, lengths)

val_test_split = int(len(rem_dataset)*0.5)

val_test_lengths = [val_test_split, len(rem_dataset) - val_test_split]
val_dataset, test_dataset = random_split(rem_dataset, val_test_lengths)


print("Length of train dataset:", len(train_dataset))
print("Length of val dataset:", len(val_dataset))
print("Length of test dataset:", len(test_dataset))

Length of train dataset: 62763
Length of val dataset: 7845
Length of test dataset: 7846


## Create dataloaders for the train, validation and test datasets

In [10]:
from torch.utils.data import DataLoader

def load_data(train_dataset, val_dataset, test_dataset, collate_fn):
    
    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn)

    
    return train_loader, val_loader, test_loader

train_loader, val_loader, test_loader = load_data(train_dataset, val_dataset, test_dataset, collate_fn)

## Create Bi-directional LSTM model 


In [11]:
class LSTM(nn.Module):
    def __init__(self):
        super().__init__()
        #self.lstm = nn.LSTM(input_size=58, hidden_size=58, bidirectional= True, batch_first=True, num_layers=3)
        self.lstm = nn.LSTM(input_size=90, hidden_size=90, bidirectional=True, batch_first=True, num_layers=1)
        self.tanh = nn.Tanh()
        self.lstmdropout = nn.Dropout()
        
        #self.fc1 = nn.Linear(8233, 256)
        #self.fc1 = nn.Linear(5473, 256)
        self.fc1 = nn.Linear(180, 50)
        self.fc2 = nn.Linear(50, 10)
        #self.fc2 = nn.Linear(116, 10)
        self.fc3 = nn.Linear(10, 2)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        batch_size = x.shape[0]
        #print('batch_size', batch_size)
        #print('x shape', x.shape)
        output,(h,c) = self.lstm(x)
        #print('output shape', output.shape)
        #print('h shape', h.shape)
        #print('h shape', h[0].shape,h[-1].shape)
        finaloutput = torch.concat([h[0],h[-1]],dim=1)
        #print('finaloutput shape',finaloutput.shape)
        
        #print(finaloutput.shape)
        finaloutput = self.lstmdropout(self.tanh(finaloutput))
        #print('after forward', finaloutput.shape)
        
            
        finaloutput = self.sigmoid(self.fc3(self.fc2(self.fc1(finaloutput))))
        #print('after sigmoid', finaloutput.shape)
        #for i in range(48):
        #    print(finaloutput[1,i,0],finaloutput[1,i,1])
        
        #print(finaloutput[0,1,0],finaloutput[0,1,1])
        #print(finaloutput[0,2,0],finaloutput[0,2,1])
        
        #print(finaloutput.shape)
        #finaloutput = torch.squeeze(finaloutput, 1)
        return finaloutput
    
model = LSTM() 

In [12]:
model

LSTM(
  (lstm): LSTM(90, 90, batch_first=True, bidirectional=True)
  (tanh): Tanh()
  (lstmdropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=180, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=2, bias=True)
  (sigmoid): Sigmoid()
)

## Print number of parameters used by the model

In [13]:
sum(p.numel() for p in model.parameters() if p.requires_grad) 

140622

## Create Adam optimizer and Cross Entropy Loss function

In [14]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()

## Model evaluation function

In [15]:
from sklearn.metrics import *

def eval_model(model, val_loader):
    model.eval()
    y_pred = []
    y_true = torch.LongTensor()
    model.eval()
    for x, y in val_loader:
        y_hat = model(x)
        #y_score = torch.cat((y_score,  y_hat.detach().to('cpu')), dim=0)
        y_hat = (y_hat > 0.5).int()
        #print(y_hat.shape)
        for i in range(len(y_hat)):
            if y_hat[i][0] == 1:
                y_pred.append(0)
            else:
                y_pred.append(1)
        #y_pred = torch.cat((y_pred,  y_hat.detach().to('cpu')), dim=0)
        y_true = torch.cat((y_true, y.detach().to('cpu')), dim=0)
    
    y_pred_t = torch.Tensor(y_pred)
    #print(y_pred_t)
    #print(y_true)
    acc = accuracy_score(y_true, y_pred_t)
    precision = precision_score(y_true, y_pred_t)
    recall = recall_score(y_true, y_pred_t)
    roc_auc = roc_auc_score(y_true, y_pred_t)
    
    return acc, precision, recall, roc_auc

## Unit test the model

In [16]:
a, p, r, roc_auc = eval_model(model, val_loader)
print(a, p, r, roc_auc)

0.48808158062460166 0.48808158062460166 1.0 0.5


## Training function 

In [17]:
#Training
def train(model, train_loader, val_loader, n_epochs):
    
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for x, y in train_loader:
            #print(len(x))
            optimizer.zero_grad()
            y_hat = model(x)
            #print(y_hat)
            #print(y)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss))
        a, p, r, roc_auc = eval_model(model, val_loader)
        print('Epoch: {} \t Validation a: {:.3f}, p:{:.3f}, r: {:.3f}, roc_auc: {:.3f}'
              .format(epoch+1, a, p, r, roc_auc))
        

## Call the training function for 10 epochs

In [18]:
import time

start_time_nanosec = time.time_ns()
print('start_time_nanosec', start_time_nanosec)

n_epochs = 10
train(model, train_loader, val_loader, n_epochs)

end_time_nanosec = time.time_ns()
print('end_time_nanosec', end_time_nanosec)


print('total time in seconds', (end_time_nanosec - start_time_nanosec)/1000000000)

start_time_nanosec 1651188017196974000
Epoch: 1 	 Training Loss: 0.567076
Epoch: 1 	 Validation a: 0.772, p:0.743, r: 0.816, roc_auc: 0.773
Epoch: 2 	 Training Loss: 0.530228
Epoch: 2 	 Validation a: 0.796, p:0.838, r: 0.722, roc_auc: 0.795
Epoch: 3 	 Training Loss: 0.523555
Epoch: 3 	 Validation a: 0.811, p:0.866, r: 0.725, roc_auc: 0.809
Epoch: 4 	 Training Loss: 0.521005
Epoch: 4 	 Validation a: 0.799, p:0.782, r: 0.815, roc_auc: 0.799
Epoch: 5 	 Training Loss: 0.511555
Epoch: 5 	 Validation a: 0.795, p:0.765, r: 0.837, roc_auc: 0.796
Epoch: 6 	 Training Loss: 0.511389
Epoch: 6 	 Validation a: 0.801, p:0.787, r: 0.812, roc_auc: 0.801
Epoch: 7 	 Training Loss: 0.509478
Epoch: 7 	 Validation a: 0.821, p:0.859, r: 0.756, roc_auc: 0.819
Epoch: 8 	 Training Loss: 0.497614
Epoch: 8 	 Validation a: 0.811, p:0.806, r: 0.807, roc_auc: 0.811
Epoch: 9 	 Training Loss: 0.495311
Epoch: 9 	 Validation a: 0.817, p:0.818, r: 0.803, roc_auc: 0.817
Epoch: 10 	 Training Loss: 0.496725
Epoch: 10 	 Vali

## Evaluate the model on the testing data

In [19]:
a, p, r, roc_auc = eval_model(model, test_loader)
print('Epoch: {} \t Testing a: {:.3f}, p:{:3f}, r: {:.3f}, roc_auc: {:.3f}'
              .format(1, a, p, r, roc_auc))

Epoch: 1 	 Testing a: 0.824, p:0.852128, r: 0.784, roc_auc: 0.824
